In [30]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
# import matplotlib.pyplot as plt
# import ast
from sklearn.preprocessing import MultiLabelBinarizer
from collections import Counter
import numpy as np
import re
from textwrap import wrap
from jinja2 import Template

In [31]:
filename = "sociotechnical_alignment_papers.csv"
df = pd.read_csv(filename, sep=';', keep_default_na=True)
df = df.dropna(subset=['subtopic'])
df

,ID,title,abstract,alignment,social aspect,modality,type,subtopic,notes
0,kravchenko-etal-2025-ualign,{UA}lign: {LLM} Alignment Benchmark for the {U...,"This paper introduces UAlign, the comprehensiv...",1,1,Text,Evaluation,moral,"While ""moral"" and ""ethical"" have slightly diff..."
1,liu-etal-2025-smaller,Smaller Large Language Models Can Do Moral Sel...,Self-correction is one of the most amazing eme...,1,1,Text,Evaluation;Mitigation,moral;safety,NaN
2,yu-etal-2025-diverse,Diverse {AI} Feedback For Large Language Model...,Recent advances in large language models (LLMs...,1,1,Text,Training;Evaluation,General,NaN
3,chen-etal-2025-instructioncp,{I}nstruction{CP}: A Simple yet Effective Appr...,The rapid development of large language models...,1,1,Text,Training;Evaluation;Mitigation,safety;multilingual,NaN
4,mittal-etal-2025-protect,{PROTECT}: Policy-Related Organizational Value...,"This paper presents PROTECT, a novel policy-dr...",1,1,Text,Training;Evaluation,ethical,NaN
...,...,...,...,...,...,...,...,...,...
286,yao-etal-2024-value,Value {FULCRA}: Mapping Large Language Models ...,Value alignment is crucial for the responsible...,1,1,Text,Training,value,NaN
287,jones-etal-2024-multi,A Multi-Aspect Framework for Counter Narrative...,Counter narratives - informed responses to hat...,1,1,Text,Evaluation,toxicity,NaN
288,zhan-etal-2024-removing,Removing {RLHF} Protections in {GPT}-4 via Fin...,As large language models (LLMs) have increased...,1,1,Text,Evaluation,safety,NaN
289,du-etal-2024-zhujiu,{Z}hu{J}iu-Knowledge: A Fairer Platform for Ev...,The swift advancement in large language models...,1,1,Text,Evaluation,General,NaN


In [17]:
years = []
for item in df['ID']:
    obj = re.search('^.+?\-((?:20)\d{2})\-', item)
    year = obj.group(1)
    years.append(year)
if 'year' not in df.columns:
    df.insert(loc=1, column='year', value=years)

In [18]:
df

,ID,year,title,abstract,alignment,social aspect,modality,type,subtopic,notes
0,kravchenko-etal-2025-ualign,2025,{UA}lign: {LLM} Alignment Benchmark for the {U...,"This paper introduces UAlign, the comprehensiv...",1,1,Text,Evaluation,moral,"While ""moral"" and ""ethical"" have slightly diff..."
1,liu-etal-2025-smaller,2025,Smaller Large Language Models Can Do Moral Sel...,Self-correction is one of the most amazing eme...,1,1,Text,Evaluation;Mitigation,moral;safety,NaN
2,yu-etal-2025-diverse,2025,Diverse {AI} Feedback For Large Language Model...,Recent advances in large language models (LLMs...,1,1,Text,Training;Evaluation,General,NaN
3,chen-etal-2025-instructioncp,2025,{I}nstruction{CP}: A Simple yet Effective Appr...,The rapid development of large language models...,1,1,Text,Training;Evaluation;Mitigation,safety;multilingual,NaN
4,mittal-etal-2025-protect,2025,{PROTECT}: Policy-Related Organizational Value...,"This paper presents PROTECT, a novel policy-dr...",1,1,Text,Training;Evaluation,ethical,NaN
...,...,...,...,...,...,...,...,...,...,...
286,yao-etal-2024-value,2024,Value {FULCRA}: Mapping Large Language Models ...,Value alignment is crucial for the responsible...,1,1,Text,Training,value,NaN
287,jones-etal-2024-multi,2024,A Multi-Aspect Framework for Counter Narrative...,Counter narratives - informed responses to hat...,1,1,Text,Evaluation,toxicity,NaN
288,zhan-etal-2024-removing,2024,Removing {RLHF} Protections in {GPT}-4 via Fin...,As large language models (LLMs) have increased...,1,1,Text,Evaluation,safety,NaN
289,du-etal-2024-zhujiu,2024,{Z}hu{J}iu-Knowledge: A Fairer Platform for Ev...,The swift advancement in large language models...,1,1,Text,Evaluation,General,NaN


In [19]:
subtpcs = df['subtopic'].apply(lambda x: [i.lower() for i in x.split(';') if x != [] and x != '' and i != [] and i != ''])
labels_list = subtpcs.to_list()
print(labels_list)

types = []
for i in subtpcs:
    for x in i:
        types.append(x.lower())
# types = subtpcs.to_list().join.unique
types = set(types)
print(types)
print(len(types))


[['moral'], ['moral', 'safety'], ['general'], ['safety', 'multilingual'], ['ethical'], ['ethical'], ['social'], ['legal'], ['multilingual'], ['moral'], ['social'], ['opinions', 'demographics'], ['safety', 'personalization'], ['safety'], ['moral', 'cultural'], ['safety', 'diversity'], ['factuality'], ['cultural', 'multilingual'], ['social'], ['safety', 'toxicity'], ['social'], ['cultural'], ['cultural', 'safety'], ['cultural', 'bias'], ['personalization', 'diversity'], ['factuality'], ['general'], ['cultural'], ['general'], ['general'], ['general'], ['social'], ['cultural'], ['ethical'], ['cultural'], ['personalization'], ['safety'], ['general'], ['safety'], ['general'], ['demographics'], ['demographics'], ['general'], ['general'], ['safety'], ['general'], ['general'], ['personalization'], ['general'], ['safety'], ['general'], ['cultural'], ['moral'], ['cultural'], ['safety'], ['safety'], ['safety'], ['general'], ['general'], ['safety'], ['general'], ['cultural'], ['general'], ['diversi

In [20]:
# Extract labels
# labels_list = df['subtopic'].tolist()
# print(labels_list)

# Get unique labels
mlb = MultiLabelBinarizer()
y_binary = mlb.fit_transform(subtpcs)
unique_labels = mlb.classes_
n_labels = len(unique_labels)
unique_years = sorted(list(set(df['year'].to_list())))

print(f"Found {len(types)} unique types: {list(types)}")
print(f"Found {n_labels} unique labels: {list(unique_labels)}")
print(f"Total items: {len(df)}")
print(f"Paper years: {unique_years}")

Found 25 unique types: ['offensiveness', 'factuality', 'demographics', 'opinions', 'safety', 'ethical', 'personalization', 'moral', 'diversity', 'culture', 'humor', 'social', 'faithfulness', 'legal', 'value', 'political', 'sexism', 'toxicity', 'length', 'general', 'bias', 'language', 'cultural', 'hate', 'multilingual']
Found 25 unique labels: ['bias', 'cultural', 'culture', 'demographics', 'diversity', 'ethical', 'factuality', 'faithfulness', 'general', 'hate', 'humor', 'language', 'legal', 'length', 'moral', 'multilingual', 'offensiveness', 'opinions', 'personalization', 'political', 'safety', 'sexism', 'social', 'toxicity', 'value']
Total items: 281
Paper years: ['2022', '2023', '2024', '2025']


In [21]:
# Separate scatter plot for each year
angles = np.linspace(0, 2 * np.pi, n_labels, endpoint=False)
cluster_centers = {label: np.array([np.cos(angle), np.sin(angle)]) * 3 
                   for label, angle in zip(unique_labels, angles)}

# Position each point based on its labels
positions = []
for lbls in labels_list:
    if len(lbls) == 1:
        # Single label: near cluster center with small jitter
        center = cluster_centers[lbls[0]]
        offset = np.random.randn(2) * 0.3
        pos = center + offset
    else:
        # Multiple labels: average of cluster centers
        centers = np.array([cluster_centers[lbl] for lbl in lbls if lbl != ''])
        if len(centers) > 0:
            center = centers.mean(axis=0)
            offset = np.random.randn(2) * 0.2
            pos = center + offset
        else:
            pos = np.array([0, 0])  # Default position if all labels are empty
    positions.append(pos)

positions = np.array(positions)

In [22]:
# angles = np.linspace(0, 2 * np.pi, n_labels, endpoint=False)
# cluster_centers = {label: np.array([np.cos(angle), np.sin(angle)]) * 3 
#                    for label, angle in zip(unique_labels, angles)}

# # Initial positioning
# positions = []
# for lbls in labels_list:
#     if len(lbls) == 1:
#         center = cluster_centers[lbls[0]]
#         offset = np.random.randn(2) * 0.3
#         pos = center + offset
#     else:
#         centers = np.array([cluster_centers[lbl] for lbl in lbls if lbl != ''])
#         if len(centers) > 0:
#             center = centers.mean(axis=0)
#             offset = np.random.randn(2) * 0.2
#             pos = center + offset
#         else:
#             pos = np.array([0, 0])
#     positions.append(pos)

# positions = np.array(positions)

# # Iteratively push overlapping points apart
# min_distance = 0.4
# n_iterations = 50
# repulsion_strength = 0.1

# for iteration in range(n_iterations):
#     forces = np.zeros_like(positions)
    
#     for i in range(len(positions)):
#         for j in range(i + 1, len(positions)):
#             diff = positions[i] - positions[j]
#             dist = np.linalg.norm(diff)
            
#             if dist < min_distance and dist > 0:
#                 # Calculate repulsion force
#                 force = (diff / dist) * repulsion_strength * (min_distance - dist)
#                 forces[i] += force
#                 forces[j] -= force
    
#     # Apply forces
#     positions += forces

In [23]:
color_palette = px.colors.qualitative.Set3
if len(unique_labels) > len(color_palette):
    color_palette = px.colors.sample_colorscale("turbo", [n/(len(unique_labels)-1) for n in range(len(unique_labels))])
label_colors = {label: color_palette[i % len(color_palette)] for i, label in enumerate(unique_labels)}

In [24]:
# Create subplots - one for each year
fig = make_subplots(
    rows=1, cols=len(unique_years),
    subplot_titles=[f'Year: {year}' for year in unique_years],
    horizontal_spacing=0.05
)

num_indices = 0
# Add traces for each year
for idx, year in enumerate(unique_years):
    col = idx + 1
    
    # # First, add all points in gray (background)
    # fig.add_trace(
    #     go.Scatter(
    #         x=positions[:, 0],
    #         y=positions[:, 1],
    #         mode='markers',
    #         marker=dict(size=8, color='lightgray', opacity=0.3),
    #         showlegend=False,
    #         hoverinfo='skip',
    #         name='Background'
    #     ),
    #     row=1, col=col
    # )
    
    # Then add each cluster for this year
    for label in unique_labels:
        indices = [i for i, lbls in enumerate(labels_list) if label in lbls and years[i] == year]
        num_indices += len(indices)
        
        if len(indices) > 0:
            cluster_points = positions[indices]
            
            # Create hover text with paper IDs and all labels
            hover_texts = []
            for i in indices:
                paper = "<br>".join(wrap(df.iloc[i]['title'], width=50))
                all_labels = ', '.join(labels_list[i])
                paper_id = df.iloc[i]['ID']
                hover_texts.append(f"Paper: {paper}<br>ID: {paper_id}<br>Subtopics: {all_labels}<br>Year: {year}")
            
            fig.add_trace(
                go.Scatter(
                    x=cluster_points[:, 0],
                    y=cluster_points[:, 1],
                    mode='markers',
                    marker=dict(
                        size=12,
                        color=label_colors[label],
                        opacity=0.7,
                        line=dict(color='black', width=1)
                    ),
                    name=f'{label} ({len(indices)})',
                    legendgroup=label,
                    showlegend=(idx == 0),  # Only show legend for first subplot
                    hovertext=hover_texts,
                    hoverinfo='text'
                ),
                row=1, col=col
            )
    
    # Update axes for this subplot
    fig.update_xaxes(
        range=[-4.5, 4.5],
        # title_text='Dimension 1',
        title_text='',
        showgrid=True,
        gridcolor='lightgray',
        row=1, col=col
    )
    fig.update_yaxes(
        range=[-4.5, 4.5],
        # title_text='Dimension 2' if col == 1 else '',
        title_text='',
        showgrid=True,
        gridcolor='lightgray',
        scaleanchor=f"x{col if col > 1 else ''}",
        scaleratio=1,
        row=1, col=col
    )

# Update overall layout
fig.update_layout(
    height=600,
    width=700 * len(unique_years),
    title_text="Multi-label Clusters by Year",
    title_font_size=16,
    hovermode='closest',
    template='plotly_white',
    # showlegend=True,
    # legend=dict(
    #     yanchor="top",
    #     y=0.99,
    #     xanchor="left",
    #     x=1.01
    # )
)

fig.show()

In [25]:
hover_texts = []
indices = [i for i, lbls in enumerate(labels_list) if label in lbls]
for i in indices:
    paper = "<br>".join(wrap(df.iloc[i]['title'], width=50))
    all_labels = ', '.join(labels_list[i])
    paper_id = df.iloc[i]['ID']
    hover_texts.append(f"Paper: {paper}<br>ID: {paper_id}<br>Subtopics: {all_labels}<br>Year: {year}")

cluster_points = positions[indices]

all_fig = go.Figure()

for idx, year in enumerate(unique_years):
    col = idx + 1
    
    # Then add each cluster for this year
    for label in unique_labels:
        indices = [i for i, lbls in enumerate(labels_list) if label in lbls]
        num_indices += len(indices)
        
        if len(indices) > 0:
            cluster_points = positions[indices]
            
            # Create hover text with paper IDs and all labels
            hover_texts = []
            for i in indices:
                paper = "<br>".join(wrap(df.iloc[i]['title'], width=50))
                all_labels = ', '.join(labels_list[i])
                paper_id = df.iloc[i]['ID']
                hover_texts.append(f"Paper: {paper}<br>ID: {paper_id}<br>Subtopics: {all_labels}<br>Year: {year}")
            
            all_fig.add_trace(
                go.Scatter(
                    x=cluster_points[:, 0],
                    y=cluster_points[:, 1],
                    mode='markers',
                    marker=dict(
                        size=12,
                        color=label_colors[label],
                        opacity=0.7,
                        line=dict(color='black', width=1)
                    ),
                    name=f'{label} ({len(indices)})',
                    legendgroup=label,
                    showlegend=(idx == 0),  # Only show legend for first subplot
                    hovertext=hover_texts,
                    hoverinfo='text'
                )
            )
    
# Update axes for this subplot
all_fig.update_layout(
    height=800,
    width=1000,
    title_text="Multi-label Clusters (All Years)",
    title_font_size=16,
    hovermode='closest',
    template='plotly_white',
    xaxis=dict(
        range=[-4.5, 4.5],
        title='',
        showgrid=True,
        gridcolor='lightgray'
    ),
    yaxis=dict(
        range=[-4.5, 4.5],
        title='',
        showgrid=True,
        gridcolor='lightgray',
        scaleanchor="x",
        scaleratio=1
    )
)

all_fig.show()

In [26]:
# Create figure
slider_fig = go.Figure()

# Add invisible legend traces first
for label in unique_labels:
    slider_fig.add_trace(
        go.Scatter(
            x=[None],  # No actual data
            y=[None],
            mode='markers',
            marker=dict(
                size=10,
                color=label_colors[label],
                line=dict(color='black', width=1)
            ),
            legendgroup=label,
            showlegend=True,
            name=label
        )
    )

num_legend_traces = len(unique_labels)

# Add traces for each label and year combination
for year in unique_years:
    for label in unique_labels:
        indices = [i for i, lbls in enumerate(labels_list) if label in lbls and years[i] == year]
        
        if len(indices) > 0:
            cluster_points = positions[indices]
            
            # Create hover text
            hover_texts = []
            for i in indices:
                paper = "<br>".join(wrap(df.iloc[i]['title'], width=50))
                all_labels = ', '.join(labels_list[i])
                paper_id = df.iloc[i]['ID']
                hover_texts.append(f"Paper: {paper}<br>ID: {paper_id}<br>Subtopics: {all_labels}<br>Year: {year}")
            
            slider_fig.add_trace(
                go.Scatter(
                    x=cluster_points[:, 0],
                    y=cluster_points[:, 1],
                    mode='markers',
                    marker=dict(
                        size=12,
                        color=label_colors[label],
                        opacity=0.7,
                        line=dict(color='black', width=1)
                    ),
                    name=f'{label}',
                    legendgroup=label,
                    showlegend=False,  # Don't show in legend (using invisible traces)
                    hovertext=hover_texts,
                    hoverinfo='text',
                    visible=False  # Start with all hidden
                )
            )

# Create slider steps
steps = []

# Build visibility mapping for each year
trace_idx = num_legend_traces  # Start after legend traces
year_trace_map = {}  # Maps year to list of trace indices

for year in unique_years:
    year_trace_map[year] = []
    for label in unique_labels:
        indices = [i for i, lbls in enumerate(labels_list) if label in lbls and years[i] == year]
        if len(indices) > 0:
            year_trace_map[year].append(trace_idx)
            trace_idx += 1

# Add step for each year
for year in unique_years:
    # Create visibility array: legend traces always visible, data traces only for this year
    visible = [True] * num_legend_traces  # Legend traces always visible
    
    # Add visibility for all data traces
    total_data_traces = len(slider_fig.data) - num_legend_traces
    visible.extend([False] * total_data_traces)
    
    # Set visible=True only for this year's traces
    for trace_idx in year_trace_map[year]:
        visible[trace_idx] = True
    
    steps.append(dict(
        method="update",
        args=[{"visible": visible},
            #   {"title": f"Multi-label Clusters - Year {year}"}
        ],
        label=str(year)
    ))

if len(steps) > 0:
    first_step_visible = steps[0]['args'][0]['visible']
    for i, trace in enumerate(slider_fig.data):
        trace.visible = first_step_visible[i]

# # Add "All Years" as first step
# all_visible = [True] * len(slider_fig.data)
# steps.insert(0, dict(
#     method="update",
#     args=[{"visible": all_visible},
#           {"title": "Multi-label Clusters (All Years)"}],
#     label="All"
# ))

# Create slider
sliders = [dict(
    active=0,
    yanchor="top",
    y=-0.1,
    xanchor="left",
    x=0.0,
    currentvalue=dict(
        prefix="Year: ",
        visible=True,
        xanchor="center"
    ),
    pad=dict(b=10, t=50),
    len=0.9,
    steps=steps
)]

# Update layout with slider
slider_fig.update_layout(
    height=700,
    width=900,
    title_text="Multi-label Clusters for each Year",
    title_font_size=16,
    hovermode='closest',
    template='plotly_white',
    sliders=sliders,
    xaxis=dict(
        range=[-4.5, 4.5],
        title='',
        showgrid=True,
        gridcolor='lightgray'
    ),
    yaxis=dict(
        range=[-4.5, 4.5],
        title='',
        showgrid=True,
        gridcolor='lightgray',
        scaleanchor="x",
        scaleratio=1
    ),
    showlegend=True
)

slider_fig.show()

In [27]:
output_html_path=r"./docs/index.html"
input_template_path = r"./docs/template_file.html"
plotly_jinja_data = {"all_fig": all_fig.to_html(full_html=False, include_plotlyjs='cdn'),
                     "slider_fig": slider_fig.to_html(full_html=False, include_plotlyjs='cdn')}
#consider also defining the include_plotlyjs parameter to point to an external Plotly.js as described above

with open(output_html_path, "w", encoding="utf-8") as output_file:
    with open(input_template_path) as template_file:
        j2_template = Template(template_file.read())
        output_file.write(j2_template.render(plotly_jinja_data))

In [28]:
# all_fig.write_html("all_fig.html")
# slider_fig.write_html("slider_fig.html")

In [29]:
# # Source - https://stackoverflow.com/a
# # Posted by Milos K, modified by community. See post 'Timeline' for change history
# # Retrieved 2026-01-28, License - CC BY-SA 4.0

# with open('index.html', 'a') as f:
#     f.write('Visualisation of Alignment Survey')
#     f.write(all_fig.to_html(full_html=False, include_plotlyjs='cdn'))
#     f.write(slider_fig.to_html(full_html=False, include_plotlyjs=False))
